# 판별 분석 

종속 변수를 그룹으로 두고 개체가 어느 그룹에 속하는지를 판별

 ##  Fisher의 판별함수

두 집단의 공통분산공분산행렬
$$S = {n_1-1 \over n_1+n_2-2}S_1 + {n_2-1 \over n_1+n_2-2}S_2$$

Fisher's discriminant function
$$Y=b_1X_1 + b_2X_2 + ... + b_pX_p = \textbf{b'}X$$

여기서 판별함수계수 b를 추정한다

b는 $\lambda = {그룹 간 분산 \over 그룹 내 분산}$이 최대가 되도록 구해야 한다.

이를 만족하는 b는 다음과 같다.
$$\textbf{b} = \Sigma^{-1}(\textbf{$\mu$}_1-\textbf{$\mu$}_2)$$

따라서 판별함수 Y는
$$Y = (\textbf{$\mu$}_1-\textbf{$\mu$}_2)\Sigma^{-1}X$$

## 분류점

두 집단의 판별함수의 평균을 각각 구한다.
$$\bar{Y}_1 = (\bar{X}_1 - \bar{X}_2)'S^{-1}\bar{X}_1$$
$$\bar{Y}_2 = (\bar{X}_1 - \bar{X}_2)'S^{-1}\bar{X}_2$$

이를 두 중심으로 하여 그 중앙 위치를 구한다.
$$Y_c = {\bar{Y}_1+\bar{Y}_2 \over 2} = {1 \over 2}(\bar{X}_1 - \bar{X}_2)'S^{-1}(\bar{X}_1+\bar{X}_2)$$

두 집단의 크기가 다르다면
$$Y_c = {n_2\bar{Y}_1+n_1\bar{Y}_2 \over n_1+n_2}$$

## 분류계수

## 사후확률

# 예제

In [1]:
library(xlsx)
library(MASS)

In [2]:
data = read.xlsx("mvadata/alcohol.xlsx",sheetName = "alcohol")

In [3]:
head(data)

,TYPE,MEOH,ACET,BU1,MEPR,ACAL,LNPRO1
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,typeA,3,15,0.2,9,9.0,5.86
2,typeA,23,13,0.8,9,2.0,6.67
3,typeA,65,96,0.4,9,4.0,5.31
4,typeA,279,66,0.9,36,4.8,5.45
5,typeA,292,210,1.1,34,8.0,4.08
6,typeA,371,414,1.2,39,9.0,6.22


In [4]:
data.lda = lda(TYPE ~ ., data = data)

Y는 TYPE X는 모두(.)

In [5]:
data.lda

Call:
lda(TYPE ~ ., data = data)

Prior probabilities of groups:
    typeA     typeB     typeC 
0.2337662 0.3766234 0.3896104 

Group means:
           MEOH     ACET       BU1     MEPR     ACAL   LNPRO1
typeA  378.6944 218.0167  1.511111 32.06667 11.16667 6.231111
typeB  939.1379 247.3448 17.906897 30.55172 12.54138 4.883103
typeC 1035.4000 173.3667 19.620000 43.00000 13.27333 5.145667

Coefficients of linear discriminants:
                 LD1           LD2
MEOH    3.382089e-03  0.0005710473
ACET   -4.649248e-05 -0.0066573606
BU1     1.322048e-01 -0.0162598664
MEPR   -2.562255e-02  0.0533609640
ACAL   -4.048757e-02  0.0297883525
LNPRO1 -2.791911e-01  0.3894400487

Proportion of trace:
   LD1    LD2 
0.9168 0.0832 

prior probabilities of groups는 실제 관측값의 비율을 의미한다.

In [6]:
names(data.lda)

[1] "prior"   "counts"  "means"   "scaling" "lev"     "svd"     "N"      
 [8] "call"    "terms"   "xlevels"

In [7]:
pred.lda = predict(data.lda, newdata = data)

In [8]:
names(pred.lda)

[1] "class"     "posterior" "x"

posterior는 사후확률이다.

In [9]:
pred.lda$class

[1] typeA typeA typeA typeA typeA typeA typeA typeA typeA typeA typeA typeA
[13] typeA typeA typeA typeA typeA typeA typeB typeB typeB typeB typeB typeB
[25] typeB typeB typeB typeC typeB typeB typeB typeC typeC typeB typeB typeC
[37] typeB typeB typeB typeB typeB typeB typeC typeB typeC typeB typeB typeB
[49] typeC typeB typeB typeC typeC typeB typeB typeC typeB typeC typeC typeB
[61] typeC typeC typeB typeC typeC typeB typeC typeC typeC typeC typeC typeC
[73] typeC typeC typeC typeC typeC
Levels: typeA typeB typeC

In [10]:
head(pred.lda$posterior)

,typeA,typeB,typeC
1,0.9999932,4.878233e-06,1.882559e-06
2,0.9999838,1.104519e-05,5.137599e-06
3,0.9998621,1.216151e-04,1.626668e-05
4,0.9999067,3.768926e-05,5.562616e-05
5,0.9993267,5.527433e-04,1.205654e-04
6,0.9999181,7.058307e-05,1.131889e-05


예를 들어 1번 개체는 typeA일 사후확률이 0.99이므로 typeA로 판별한다.

In [11]:
confm.lda = table(data$TYPE, pred.lda$class)
confm.lda

       
        typeA typeB typeC
  typeA    18     0     0
  typeB     0    23     6
  typeC     0     9    21

관측값과 class를 교차하여 confusion matrix를 그려준다.

In [12]:
error = 1-sum(diag(confm.lda))/sum(confm.lda)
error

[1] 0.1948052

잘못 분류된 오류들을 혼돈행렬을 통해 계산한다.

## 변수 선택

In [32]:
library(klaR)

In [33]:
data.forward = greedy.wilks(TYPE~.,data = data, niveau = 0.01)

In [34]:
data.forward

Formula containing included variables: 

TYPE ~ BU1 + MEPR + MEOH
<environment: 0x55ef7a9a43a8>


Values calculated in each step of the selection procedure: 

  vars Wilks.lambda F.statistics.overall p.value.overall F.statistics.diff
1  BU1    0.2989919             86.74917    3.975561e-20         86.749173
2 MEPR    0.2489312             36.65654    3.402765e-21          7.340239
3 MEOH    0.2012539             29.49819    7.226240e-23          8.528444
  p.value.diff
1 3.975561e-20
2 1.235768e-03
3 4.692630e-04

검정 결과 유의한 변수들을 선택함 niveau에 유의수준 

In [35]:
data.fwd.lda = lda(data.forward$formula, data = data)
data.fwd.lda

Call:
lda(data.forward$formula, data = data)

Prior probabilities of groups:
    typeA     typeB     typeC 
0.2337662 0.3766234 0.3896104 

Group means:
            BU1     MEPR      MEOH
typeA  1.511111 32.06667  378.6944
typeB 17.906897 30.55172  939.1379
typeC 19.620000 43.00000 1035.4000

Coefficients of linear discriminants:
              LD1           LD2
BU1   0.149874251 -0.0313069489
MEPR -0.025447467  0.0759388778
MEOH  0.002809521  0.0002065334

Proportion of trace:
   LD1    LD2 
0.9516 0.0484 

In [36]:
pred.fwd.lda = predict(data.fwd.lda, newdata = data)

In [37]:
confm.fwd = table(data$TYPE, pred.fwd.lda$class)
confm.fwd

       
        typeA typeB typeC
  typeA    18     0     0
  typeB     0    22     7
  typeC     1    12    17

In [38]:
error = 1-sum(diag(confm.fwd))/sum(confm.fwd)
error

[1] 0.2597403

에러가 더 늘었음

## 분류함수 계수 구하기

In [47]:
source("mvadata/classFunc.r")

classFunc 소스 불러오기

In [48]:
X = data[, -1]

In [49]:
classfunc.result = classfunc.lda(X, data$TYPE)

In [50]:
names(classfunc.result)

[1] "prior"      "counts"     "means"      "scaling"    "lev"       
 [6] "svd"        "N"          "call"       "terms"      "xlevels"   
[11] "class.func"

In [52]:
classfunc.result$class.func

,typeA,typeB,typeC
constant,-2.468616e+01,-25.14175440,-29.631643758
MEOH,6.590818e-04,0.01520849,0.016407192
ACET,4.449847e-04,0.00494391,-0.003769616
BU1,-3.934206e-02,0.55665395,0.553048496
MEPR,1.868923e-01,0.03590143,0.102270552
ACAL,3.917393e-02,-0.16088059,-0.127328438
LNPRO1,6.378935e+00,4.86936954,5.341430073


In [56]:
id1 = c(1, 3, 15, 0.2, 9, 9, 5.86)

In [57]:
lda.value = id1 * classfunc.result$class.func

In [58]:
apply(lda.value, 2, sum)

typeA     typeB     typeC 
14.729774  2.499054  1.546903

첫 개체의 판별함수의 계수가 나오게 됨 이 경우는 typeA가 크므로 typeA